# The "Azure ML SDK" for SMS Spam Inference 

## Introduction

In this notebook, we will show the use of Azure ML SDK to train, deploy and consume a model through Azure ML.


Steps:

1. Create a workspace. Create an Experiment in an existing Workspace.
2. Create a Compute cluster.
3. Load the dataset.
4. Select and Train a model.
5. Configure HyperDrive.
6. Run the HyperDrive experiment usinng HyperDriveConfig.
7. Explore the results and get the best model.
8. Register the best model.
9. Deploy the best model and create endpoint.
10. Consume the endpoint.


## Azure Machine Learning SDK-specific imports

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

In [ ]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create an Azure ML experiment

Let's create an experiment named 'aml-experiment' in the workspace we just initialized.

In [ ]:
experiment_name = 'nb_lr_exp_0920_v10'
experiment = Experiment(ws, experiment_name)
experiment

## Create a Compute Cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) for your AutoML run.

In [ ]:
aml_name = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Load Data
Make sure you have uploaded the dataset to Azure ML and that the key is the same name as the dataset.

In [ ]:
key = 'UdacityPrjEmailSpamDataSet'
smsspam_ds = ws.datasets[key]
df = smsspam_ds.to_pandas_dataframe()
df.describe()

### Create workspace folders

In [ ]:
import os

experiment_folder = 'nb_lr_exp_0920_v10'
os.makedirs(experiment_folder, exist_ok=True)

## Environment

In [ ]:
%%writefile $experiment_folder/hyperdrive_env_v10.yml
name: batch_environment
dependencies:
- python=3.8.5
- scikit-learn
- pandas
- numpy
- regex
- nltk
- pip
- pip:
  - azureml-defaults

# Python script to prepare the data train the model.

In [ ]:
%%writefile $experiment_folder/train_0920_v10.py

#import libraries
import argparse, joblib, os
from azureml.core import Run

import logging
import os
import csv
import string
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from nltk.stem import SnowballStemmer

from sklearn.metrics import accuracy_score

import regex as re

import pickle
import tempfile

from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

import azureml.core
from azureml.core.dataset import Dataset
import scipy.sparse

import numpy as np
from scipy.sparse import lil_matrix, csr_matrix, issparse
import re


# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Input dataset
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# Hyperparameters
#parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
# parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

parser.add_argument('--C', type=float, default=1.0, help="indicates regularization")
parser.add_argument('--max_iter', type=int, default=50, help="Maximum number of iterations")

# Add arguments to args collection
args = parser.parse_args()

# Log Hyperparameter values
# run.log('learning_rate',  np.float(args.learning_rate))
# run.log('n_estimators',  np.int(args.n_estimators))

run.log("Regularization Strength:", np.float(args.C))
run.log("Max iterations:", np.int(args.max_iter))

 
# load the sms spam dataset -- Get the training data from the input
print("Loading SMS Spam Data...")
df = run.input_datasets['training_data'].to_pandas_dataframe() 

#--------------------------Prepare Data-------------------------------------------------
# Cleanup and Prepare Data # Find and eliminate stop words 
nltk.download('stopwords')
stop_words= set(stopwords.words("english"))
stop_words.update(['https', 'http', 'amp', 'CO', 't', 'u', 'new', "I'm", "would"])


spam = df.query("v1=='spam'").v2.str.cat(sep=" ")
ham = df.query("v1=='ham'").v2.str.cat(sep=" ")

# convert spam to 1 and ham to 0
df = df.replace('spam', 1)
df = df.replace('ham', 0)

# Clean the text
def clean_text(text):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = text.replace('.', '')
    text = whitespace.sub(' ', text)
    text = web_address.sub('', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    return text.lower()

df.v2 = [clean_text(item) for item in df.v2]

#---------------------More Data Prep-----------#
df = df.drop(['Column3', 'Column4', 'Column5'], axis = 1)

df_msg_copy = df['v2'].copy()

def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

df_msg_copy = df_msg_copy.apply(text_preprocess)

def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

df_msg_copy = df_msg_copy.apply(stemmer)
vectorizer = TfidfVectorizer(stop_words='english')
train_corpus = vectorizer.fit_transform(df_msg_copy)


# Split Train and Test
xTrain, xTest, yTrain, yTest = train_test_split(train_corpus, df.v1, test_size=0.3, random_state=20)

# --------------------------End Prepare Data--------------------------------------------


# --------------------------Start Training----------------------------------------------
# Train a Logistic Regression classification model without the specified hyperparameters
print('Training a classification model')

model = LogisticRegression(solver='liblinear', penalty='l1', C=args.C, max_iter=args.max_iter )
model.fit(xTrain, yTrain)
pred = model.predict(xTest)
acc = accuracy_score(yTest,pred)

# ---------------------------End Training------------------------------------------------

run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(xTest)
auc = roc_auc_score(yTest,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model_v10.pkl')

run.complete()


## HyperDrive Configuration

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric

# Run a hyperparameter tuning experiment

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
hyper_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/hyperdrive_env_v10.yml")

# Early termination policy
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2)

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='train_0920_v10.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', smsspam_ds.as_named_input('training_data')],
                                environment=hyper_env,
                                compute_target = aml_compute)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--C': choice(0.01, 0.1, 1.0),
        '--max_iter' : choice(10, 50)
        # '--C': choice(1.0),
        # '--max_iter' : choice(10)
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=early_termination_policy, # Banditpolicy early stopping policy
                          primary_metric_name='Accuracy', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=36, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# Run the experiment
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

# Select the best performing run

In [ ]:
# Print all child runs, sorted by the primary metric
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)


In [ ]:
# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

## Register Best Model

Now that you've found the best run, you can register the model it trained.

In [ ]:
# Register best model best model
reg_model = best_run.register_model(model_name='sms-spam-hd-0920-v10-model',
                                        model_path='outputs/model_v10.pkl', 
                                        tags={'Method':'LogisticRegression Hyperdrive'}, 
                                        # sample_input_dataset = smsspam_ds,
                                        properties={'Accuracy': best_run_metrics['Accuracy'],
                                                    'AUC': best_run_metrics['AUC']})
print(reg_model)

In [26]:
reg_model.id

'sms-spam-hd-0920-v10-model:2'

In [27]:
for child_run in run.get_children():
    print(child_run,"\n")

Run(Experiment: nb_lr_exp_0920_v10,
Id: HD_ec2e2bbc-f3a6-4355-80ce-13d828b59e17_5,
Type: azureml.scriptrun,
Status: Completed) 

Run(Experiment: nb_lr_exp_0920_v10,
Id: HD_ec2e2bbc-f3a6-4355-80ce-13d828b59e17_4,
Type: azureml.scriptrun,
Status: Completed) 

Run(Experiment: nb_lr_exp_0920_v10,
Id: HD_ec2e2bbc-f3a6-4355-80ce-13d828b59e17_2,
Type: azureml.scriptrun,
Status: Completed) 

Run(Experiment: nb_lr_exp_0920_v10,
Id: HD_ec2e2bbc-f3a6-4355-80ce-13d828b59e17_3,
Type: azureml.scriptrun,
Status: Completed) 

Run(Experiment: nb_lr_exp_0920_v10,
Id: HD_ec2e2bbc-f3a6-4355-80ce-13d828b59e17_1,
Type: azureml.scriptrun,
Status: Completed) 

Run(Experiment: nb_lr_exp_0920_v10,
Id: HD_ec2e2bbc-f3a6-4355-80ce-13d828b59e17_0,
Type: azureml.scriptrun,
Status: Completed) 



In [28]:
reg_model.name

'sms-spam-hd-0920-v10-model'

In [30]:
model_name = reg_model.name
script_file = "scripts/score_v10.py"
description = "aml SMS spam lr hd project sdk"


In [31]:
from azureml.automl.core.shared import constants
env = best_run.get_environment()

## Deploy Model as Webservice on Azure Container Instance

Run the following code to deploy the best model. You can see the state of the deployment in the Azure ML portal. This step can take a few minutes.

In [32]:
inference_config = InferenceConfig(entry_script=script_file, environment=best_run.get_environment())


In [33]:
inference_config

InferenceConfig(entry_script=scripts/score_v10.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=None, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7fbf6d9b42b0>)

In [34]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-SMS-spam-prediction"},
                                               description = 'Sample service for AutoML SMS Spam Prediction')

aci_service_name = 'smsspam-lrhd-v10-7'

## Create Web Service

In [35]:
aci_service = Model.deploy(ws, aci_service_name, [reg_model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-21 03:26:32-04:00 Creating Container Registry if not exists.
2022-09-21 03:26:32-04:00 Registering the environment.
2022-09-21 03:26:33-04:00 Use the existing image.
2022-09-21 03:26:34-04:00 Submitting deployment to compute.
2022-09-21 03:26:39-04:00 Checking the status of deployment smsspam-lrhd-v10-7..
2022-09-21 03:28:46-04:00 Checking the status of inference endpoint smsspam-lrhd-v10-7.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Service test

***REST Endpoint***

In [36]:
scoring_url=aci_service.scoring_uri
print(scoring_url)

http://f179cabe-8ce5-48b1-b153-924873c6816b.centralus.azurecontainer.io/score


In [37]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

data =  {
  "data": [
    {
      "v2": "Is that seriously how you spell his name"
    }
  ],
  "method": "predict"
}

body = str.encode(json.dumps(data))

# url = 'http://85c032ba-bdb1-4060-801d-cd7a6bc543a8.centralus.azurecontainer.io/score'
url = scoring_url
api_key = '' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"result\\": [0]}"'


In [38]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

data =  {
  "data": [
    {
      "v2": "XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here"
    }
  ],
  "method": "predict"
}

body = str.encode(json.dumps(data))

# url = 'http://85c032ba-bdb1-4060-801d-cd7a6bc543a8.centralus.azurecontainer.io/score'
url = scoring_url
api_key = '' # Replace this with the API key for the web service

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"result\\": [1]}"'


## Delete Service

In [39]:
aci_service.delete

<bound method Webservice.delete of AciWebservice(workspace=Workspace.create(name='nahmed30-azureml-workspace', subscription_id='16bc73b5-82be-47f2-b5ab-f2373344794c', resource_group='epe-poc-nazeer'), name=smsspam-lrhd-v10-7, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://f179cabe-8ce5-48b1-b153-924873c6816b.centralus.azurecontainer.io/score, tags={'type': 'automl-SMS-spam-prediction'}, properties={'azureml.git.repository_uri': 'https://github.com/Nazeer2013/nd00333-capstone.git', 'mlflow.source.git.repoURL': 'https://github.com/Nazeer2013/nd00333-capstone.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '12f6fc1128f76313f7a9bff2fc986fb76046de43', 'mlflow.source.git.commit': '12f6fc1128f76313f7a9bff2fc986fb76046de43', 'azureml.git.dirty': 'True', 'hasInferenceSchema': 'True', 'hasHttps': 'False', 'authEnabled': 'False'}, created_by={'userObjectId': 'a8930881-263c-498d-8975-58e6a0c28f2c', 'userPuId':